## G1 - Salute e Reddito

In [1]:
# Import librerie per analisi dati (Pandas) e dati Istat
import os
import pandas as pd
import numpy as np
from IPython.core.display import HTML
import istat
import jsonstat

# cache dir per velocizzare analisi in locale
cache_dir = os.path.abspath(os.path.join("..", "tmp/od_la_grande_fuga", "istat_cached"))
istat.cache_dir(cache_dir)
istat.lang(0)  # lingua italiano
print("cache_dir is '{}'".format(istat.cache_dir()))

cache_dir is '/Users/cesar/Documents/opendata/tmp/od_la_grande_fuga/istat_cached'


In [2]:
# Lista Aree
# istat.areas()
istat_area_pop = istat.area(8)

## Speranza di vita alla nascita

In [3]:
# Lista Dataset - Popolazione -> KPI: Speranza di vita alla nascita
istat_dataset_popolazione = istat_area_pop.dataset('DCIS_INDDEMOG1')
# istat_dataset_popolazione

In [4]:
# Lista di tutte le Dimensioni
#istat_dataset_popolazione.dimensions()

In [5]:
# Speranza di Vita Maschi
spec1 = {
    #"Territorio":1,                 # 1 -> Italia
    #"Anno":2167,                   # ALL
    "Tipo indicatore":75            # 75 -> Speranza di vita alla nascita - maschi
}

collection1 = istat_dataset_popolazione.getvalues(spec1)
ds = collection1.dataset(0)
df_m = ds.to_data_frame('Territorio')

In [6]:
# Speranza di Vita Femmine
spec2 = {
    #"Territorio":1,                # 1 -> Italia
    #"Anno":2167,                   # ALL
    "Tipo indicatore":77            # 77 -> Speranza di vita alla nascita - femmine
}

collection2 = istat_dataset_popolazione.getvalues(spec2)
ds2 = collection2.dataset(0)
df_f = ds2.to_data_frame('Territorio')

In [7]:
df_f.count()

Anno     2085
Value    1976
dtype: int64

In [8]:
df_m.count()

Anno     2085
Value    1976
dtype: int64

In [9]:
# Calcolo Speranza di Vita alla Nascita (media tra maschi e femmine) - Territorio Italia, Nord e Sud
df_merge = pd.concat([df_m, df_f], axis=1)
df_merge.columns = ['Anno', 'Value_M', 'Anno_F', 'Value_F']
df_merge = df_merge.drop('Anno_F', 1)
df_merge.reset_index(level=0, inplace=True)

def value(x):
    return round(((x['Value_F'] + x['Value_M'])/2),1)

df_merge['Value'] = df_merge.apply(lambda x: value(x), axis=1)
df_merge = df_merge.drop('Value_F', 1)
df_merge = df_merge.drop('Value_M', 1)

df_merge=df_merge[(df_merge['Territorio']=='Italia') |
                  (df_merge['Territorio']=='Nord') |
                  (df_merge['Territorio']=='Sud')]

In [10]:
df_merge.head(2)

# writer = pd.ExcelWriter('Vita.xlsx')
# df_merge.to_excel(writer,'vita')
# writer.save()


,Territorio,Anno,Value
0,Italia,2002,80.1
1,Italia,2003,80.0


In [11]:
# Directory dove salvare i file, da utilizzare in data_visualization
dir_df = os.path.join(os.path.abspath(''),'stg')

df_merge_filename = r'df_vita.pkl'
df_merge_fullpath = os.path.join(dir_df, df_merge_filename)
df_merge.to_pickle(df_merge_fullpath)

## Reddito pro capite

In [13]:
istat_area_cn = istat.area(17)
istat_dataset_reddito = istat_area_cn.dataset('DCCN_TNA')
# istat_dataset_reddito

In [14]:
#istat_dataset_reddito.dimensions()

In [15]:
# Prodotto Interno Lordo ai prezzi di mercato per abitante

spec2 = {
#"Territorio":1,
"Tipo aggregato":27,
#"Branca di attività (NACE Rev2)":128,
#"Attività non finanziarie":3,
#"Funzione di spesa (coicop/cofog)":3,
"Valutazione":9,
#"Correzione":1,
#"Tipologia di prezzo":1,
"Edizione":2223
#"Tempo e frequenza":1920
}

collection2 = istat_dataset_reddito.getvalues(spec2)
#ds = collection.dataset(0)

In [16]:
ds = collection2.dataset(0)
df_red = ds.to_data_frame('Tempo e frequenza')
df_red.reset_index(level=0, inplace=True)
df_red = df_red[['Tempo e frequenza','Territorio', 'Value']]

In [17]:
df_red=df_red[(df_red['Territorio']=='Italia') |
                  (df_red['Territorio']=='Nord') |
                  (df_red['Territorio']=='Sud')]

In [18]:
#writer = pd.ExcelWriter('Reddito.xlsx')
#df_red.to_excel(writer,'reddito')
#writer.save()

In [19]:
df_red_filename = r'df_reddito.pkl'
df_red_fullpath = os.path.join(dir_df, df_red_filename)
df_red.to_pickle(df_red_fullpath)

In [20]:
df_red.head(2)

,Tempo e frequenza,Territorio,Value
0,1995,Italia,17327.71
1,1996,Italia,18344.68


## Numerosità popolazione

In [21]:
# Popolazione Residente
istat_dataset_residenti = istat_area_pop.dataset('DCIS_POPRES1')
istat_dataset_residenti

nr,name,nr. values,values (first 3 values)
0,Territorio,8514,"1:'Italia', 3:'Nord', 4:'Nord-ovest' ..."
1,Tipo di indicatore demografico,1,1:'popolazione al 1º gennaio'
2,Sesso,3,"1:'maschi', 2:'femmine', 3:'totale' ..."
3,Età,102,"1:'0 anni', 2:'1 anni', 3:'2 anni' ..."
4,Stato civile,5,"1:'nubile/celibe', 2:'coniugata/o', 3:'divorziata/o' ..."
5,Anno,15,"1939:'2002', 1958:'2003', 1977:'2004' ..."


In [22]:
# istat_dataset_residenti.dimensions()

In [23]:
# Popolazione Residente
spec_pr = {
    #"Territorio":1,
    "Sesso":3,
    "Stato civile":8,
    "Tipo di indicatore demografico":1,
    "Età":182,
    "Anno":2186,
}

collection_pr = istat_dataset_residenti.getvalues(spec_pr)
ds_pr = collection_pr.dataset(0)
ds_pr

pos,id,label,size,role
0,IDITTER107,Territorio,8089,


In [24]:
df_pr = ds_pr.to_data_frame('Territorio')
df_pr.reset_index(level=0, inplace=True)

In [25]:
df_pr_2015=df_pr[(df_pr['Territorio']=='Italia') |
                  (df_pr['Territorio']=='Nord') |
                  (df_pr['Territorio']=='Sud')]

## 1° Dataset - Dati per Grafico di Preston

In [26]:
# df_red -> reddito
# df_merge -> speranza di vita
# df_pr_2015 -> popolazione
df_pr_2015.rename(columns={'Value': 'Popolazione residente'}, inplace=True)
df_pr_2015

/Users/cesar/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


,Territorio,Popolazione residente
0,Italia,60795612
1,Nord,27799803
86,Sud,14149806


In [27]:
df_red_2015 = df_red[df_red['Tempo e frequenza']=='2015']
df_red_2015.drop(['Tempo e frequenza'], axis=1, inplace=True)
df_red_2015.rename(columns={'Value': 'Reddito pro capite'}, inplace=True)
df_red_2015

/Users/cesar/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/cesar/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


,Territorio,Reddito pro capite
20,Italia,27044.75
62,Nord,32964.21
461,Sud,17867.38


In [28]:
df_merge_2015 = df_merge[df_merge['Anno']=='2015']
df_merge_2015.drop(['Anno'], axis=1, inplace=True)
df_merge_2015.rename(columns={'Value': 'Speranza di vita alla nascita'}, inplace=True)
df_merge_2015

/Users/cesar/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/cesar/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


,Territorio,Speranza di vita alla nascita
13,Italia,82.3
28,Nord,82.8
1303,Sud,81.7


In [29]:
# Dataset Finale df_pr_2015, df_merge_2015,df_red_2015
result = pd.merge(df_pr_2015, df_merge_2015, on='Territorio')
df_g1 = pd.merge(result,df_red_2015, on='Territorio')
df_g1

,Territorio,Popolazione residente,Speranza di vita alla nascita,Reddito pro capite
0,Italia,60795612,82.3,27044.75
1,Nord,27799803,82.8,32964.21
2,Sud,14149806,81.7,17867.38


In [30]:
df_g1_filename = r'df_g1.pkl'
df_g1_fullpath = os.path.join(dir_df, df_g1_filename)
df_g1.to_pickle(df_g1_fullpath)